In [47]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

import requests
import json
import pymongo

In [7]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [8]:
db = client.mars_db
collection = db.items

In [48]:
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response_news = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response_news.text, 'lxml')


In [49]:
result = soup.find('div', class_='slide')
#print(results)


#for result in results_news:
news_title = result.find('div', class_='content_title').text
news_body = result.find('div', class_='rollover_description_inner').text


print(news_title)
print(news_body)
    
 



NASA Garners 7 Webby Award Nominations



Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.



In [50]:
#begin process to find featured mars image
#set up splinter to navigate to chrome browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [51]:
#Navigate to URL and click to featured image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
browser.click_link_by_partial_text('FULL IMAGE')


In [52]:
#scrape url to find featured img url
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
image_path = []
results_img = soup.find_all('img', class_='fancybox-image')
for image in results_img:
    image_path = image['src']
    print(image_path)

/spaceimages/images/mediumsize/PIA18904_ip.jpg


In [53]:
#add base_url and image_path for full_image_url
base_url = 'https://www.jpl.nasa.gov'
featured_image_url = base_url+str(image_path)
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18904_ip.jpg


In [19]:
#Begin Process to scrape Twitter for Mars weather data
url = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [20]:
results_twitter = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
mars_weather = results_twitter
print(mars_weather)


InSight sol 158 (2019-05-07) low -99.7ºC (-147.5ºF) high -21.8ºC (-7.2ºF)
winds from the SSE at 4.8 m/s (10.7 mph) gusting to 13.6 m/s (30.4 mph)
pressure at 7.50 hPapic.twitter.com/8SrPjAhpGZ


In [21]:
#Begin Process to Scrape Table from Mars facts
url = 'https://space-facts.com/mars/'

In [22]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [23]:
df = tables[0]
df.columns = ['Description', 'Values']
df.head()

,Description,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [24]:
df.set_index('Description', inplace=True)
df.head()

,Values
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [25]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr

In [26]:
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [30]:
#begin process to scrape the images of the hemispheres
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [31]:
#Navigate to URL and click to featured image
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results_img_link = soup.find_all('h3')

link_names = []

for link_name in results_img_link:
    
    link_name = soup.find('h3').text
    link_names.append(link_name)

print(link_names)

url_mars = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_mars)
browser.click_link_by_partial_text('Cerberus Hemisphere Enhanced')

#scrape url to find featured img url
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
results_mars = soup.find_all('img', class_='wide-image')
for image in results_mars:
    image_path_mars_CHE = image['src']
    print(image_path_mars_CHE)


['Cerberus Hemisphere Enhanced', 'Cerberus Hemisphere Enhanced', 'Cerberus Hemisphere Enhanced', 'Cerberus Hemisphere Enhanced']
/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg


In [32]:
mars_base_url = 'https://astrogeology.usgs.gov'
CHE_path = mars_base_url + image_path_mars_CHE
print(CHE_path)

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg


In [33]:
url_mars = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_mars)
browser.click_link_by_partial_text('Schiaparelli Hemisphere Enhanced')

#scrape url to find featured img url
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
results_mars = soup.find_all('img', class_='wide-image')
for image in results_mars:
    image_path_mars_SHE = image['src']
    print(image_path_mars_SHE)

/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg


In [34]:
mars_base_url = 'https://astrogeology.usgs.gov'
SHE_path = mars_base_url + image_path_mars_SHE
print(SHE_path)

https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg


In [35]:
url_mars = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_mars)
browser.click_link_by_partial_text('Syrtis Major Hemisphere Enhanced')

#scrape url to find featured img url
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
results_mars = soup.find_all('img', class_='wide-image')
for image in results_mars:
    image_path_mars_SMHE = image['src']
    print(image_path_mars_SMHE)

/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg


In [36]:
mars_base_url = 'https://astrogeology.usgs.gov'
SMHE_path = mars_base_url + image_path_mars_SMHE
print(SMHE_path)

https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg


In [37]:
url_mars = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_mars)
browser.click_link_by_partial_text('Valles Marineris Hemisphere Enhanced')

#scrape url to find featured img url
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
results_mars = soup.find_all('img', class_='wide-image')
for image in results_mars:
    image_path_mars_VMHE = image['src']
    print(image_path_mars_VMHE)

/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


In [38]:
mars_base_url = 'https://astrogeology.usgs.gov'
VMHE_path = mars_base_url + image_path_mars_VMHE
print(VMHE_path)

https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


In [40]:
hemisphere_image_urls = [
    
    {"title": "Valles Marineris Hemisphere", "img_url": VMHE_path},
    {"title": "Cerberus Hemisphere", "img_url": CHE_path},
    {"title": "Schiaparelli Hemisphere", "img_url": SHE_path},
    {"title": "Syrtis Major Hemisphere", "img_url": SMHE_path}    
    
    
]

In [43]:
print(hemisphere_image_urls)

[{'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}, {'title': 'Cerberus Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}]
